In [1]:
cd ../

/Users/rachelbittner/Dropbox/MARL/repos/motif


In [2]:
import motif
import os
import tempfile as tmp
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

# Make Ground Truth Seeds

In [3]:
import medleydb as mdb
import numpy as np
import mir_eval
import csv

In [4]:
mtracks = mdb.load_melody_multitracks()

In [5]:
def get_ground_truth_seeds(mtrack):
    sns.set_style("dark")
    print mtrack.track_id
    mtrack.load_melody_annotations()
    data = np.array(mtrack.melody3_annotation)
    mel3 = data[:, 1:]
    times = data[:, 0]
    n_times, n_mels = mel3.shape

    c_index = []
    c_times = []
    c_freqs = []

    seeds = []
    idx = 0
    plt.figure()
    for mel_num in range(n_mels):
        nonzero = mel3[:, mel_num] > 0
        nonzero_diff = np.diff(nonzero.astype(int))

        contour_starts = list(np.where(nonzero_diff == 1)[0] + 1)
        contour_ends = list(np.where(nonzero_diff == -1)[0] + 1)

        if contour_ends[0] < contour_starts[0]:
            contour_starts.insert(0, 0)

        if len(contour_ends) < len(contour_starts):
            contour_ends.append(len(nonzero_diff))

        for s, e in zip(contour_starts, contour_ends):

            t = times[s:e]
            f = mel3[s:e, mel_num]

            contour_diff = np.abs(np.diff(mir_eval.melody.hz2cents(f)))

            splits = list(np.where(contour_diff > 50)[0] + 1)

            if len(splits) > 0:
                if splits[-1] == len(contour_diff):
                    splits = splits[:-1]
                split_starts = [0] + splits
                split_ends = splits + [len(t) - 1]

                for ss, se in zip(split_starts, split_ends):

                    tt = t[ss:se]
                    ii = idx * np.ones(tt.shape)
                    idx += 1
                    ff = f[ss:se]
                    c_index.append(ii)
                    c_times.append(tt)
                    c_freqs.append(ff)
                    seed_idx = int(round(len(tt)/2))
                    seeds.append([int(round(44100*tt[seed_idx])), ff[seed_idx]])
                    plt.plot(tt, ff)
            else:
                i = idx * np.ones(t.shape)
                idx += 1
                c_index.append(i)
                c_times.append(t)
                c_freqs.append(f)
                seed_idx = int(round(len(t)/2))
                seeds.append([int(round(44100*t[seed_idx])), f[seed_idx]])
                plt.plot(t, f)

    plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
    plt.tick_params(axis='y', which='both', bottom='off', top='off', labelleft='off' )
#     plt.show()
    return seeds, c_index, c_times, c_freqs

In [6]:
track_ids = []
for mtrack in mtracks:

#     output_path = '/Users/rachelbittner/Documents/HLL_Data/mdb_seeds_mel3/{}_gt_seeds.csv'.format(mtrack.track_id)
#     if os.path.exists(output_path):
#         continue

    track_ids.append(mtrack.track_id)
#     seeds, _, _, _ = get_ground_truth_seeds(mtrack)
#     plt.savefig('/Users/rachelbittner/Desktop/contour_art/{}.pdf'.format(mtrack.track_id))
#     with open(output_path, 'w') as fhandle:
#         writer = csv.writer(fhandle, delimiter=',')
#         writer.writerows(seeds)


In [7]:
track_ids

['AClassicEducation_NightOwl',
 'AimeeNorwich_Child',
 'AimeeNorwich_Flying',
 'AlexanderRoss_GoodbyeBolero',
 'AlexanderRoss_VelvetCurtain',
 'AmarLal_Rest',
 'AmarLal_SpringDay1',
 'Auctioneer_OurFutureFaces',
 'AvaLuna_Waterduct',
 'BigTroubles_Phantom',
 'BrandonWebster_DontHearAThing',
 'BrandonWebster_YesSirICanFly',
 'CelestialShore_DieForUs',
 'ChrisJacoby_BoothShotLincoln',
 'ChrisJacoby_PigsFoot',
 'ClaraBerryAndWooldog_AirTraffic',
 'ClaraBerryAndWooldog_Boys',
 'ClaraBerryAndWooldog_Stella',
 'ClaraBerryAndWooldog_TheBadGuys',
 'ClaraBerryAndWooldog_WaltzForMyVictims',
 'Creepoid_OldTree',
 'CroqueMadame_Oil',
 'CroqueMadame_Pilot',
 'Debussy_LenfantProdigue',
 'DreamersOfTheGhetto_HeavyLove',
 'EthanHein_1930sSynthAndUprightBass',
 'EthanHein_GirlOnABridge',
 'FacesOnFilm_WaitingForGa',
 'FamilyBand_Again',
 'Handel_TornamiAVagheggiar',
 'HeladoNegro_MitadDelMundo',
 'HezekiahJones_BorrowedHeart',
 'HopAlong_SisterCities',
 'InvisibleFamiliars_DisturbingWildlife',
 'JoelHe

# HLL Parameter Search

In [8]:
import random
import glob
import pandas as pd

In [90]:
n_harmonics_grid = [1, 2, 3, 4, 5, 6]
tracking_gain_grid = [8e-5, 1e-4, 3e-4, 5e-4, 8e-4, 1e-3, 2e-3, 3e-3, 5e-3]
min_contour_len_grid = [0, 10, 441, 2205, 4410, 11025, 22050, 44100, 88200, 132300, 176400, 220500, 441000]
amplitude_threshold_grid = [0, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
tracking_update_grid = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 500]

# reduced grids...
# n_harmonics_grid = [3, 4, 5, 6]
# tracking_gain_grid = [1e-3]
# min_contour_len_grid = [441, 2205, 4410, 11025, 22050]
# amplitude_threshold_grid = [1e-6, 1e-5]
# tracking_update_grid = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

def parametrize_hll():
    etr = motif.contour_extractors.hll.HLL()
    etr.n_harmonics = random.choice(n_harmonics_grid)
    etr.f_cutoff = 30
    etr.tracking_gain = random.choice(tracking_gain_grid)
    etr.min_contour_len_samples = random.choice(min_contour_len_grid)
    etr.amplitude_threshold = np.random.choice(amplitude_threshold_grid)
    etr.tracking_update_threshold = np.random.choice(tracking_update_grid)

    return etr

In [10]:
def load_melody3_multif0(fpath):
    times = []
    freqs = []
    with open(fpath, 'r') as fhandle:
        reader = csv.reader(fhandle, delimiter=',')
        for line in reader:
            times.append(float(line[0]))
            f = np.array(line[1:], dtype=float)
            f = f[f > 0]
            freqs.append(f)
    return np.array(times), freqs

In [11]:
def run_hll(etr, audio_filepath, seed_fpath):
    tmp_audio = etr._preprocess_audio(
        audio_filepath, normalize_format=True, normalize_volume=True
    )

    contours_fpath = tmp.mktemp('.csv')

    args = [
        "hll",
        "{}".format(tmp_audio),
        "{}".format(seed_fpath),
        "{}".format(contours_fpath),
        "{}".format(etr.n_harmonics),
        "{}".format(etr.f_cutoff),
        "{}".format(etr.tracking_gain),
        "{}".format(etr.min_contour_len_samples),
        "{}".format(etr.amplitude_threshold),
        "{}".format(etr.tracking_update_threshold)
    ]

    os.system(' '.join(args))

    if not os.path.exists(contours_fpath):
        raise IOError(
            "Unable to find HLL output file {}".format(contours_fpath)
        )

    c_numbers, c_times, c_freqs, c_sal = etr._load_contours(contours_fpath)

    os.remove(contours_fpath)
    os.remove(tmp_audio)

    return motif.core.Contours(
        c_numbers, c_times, c_freqs, c_sal, etr.sample_rate, audio_filepath
    )

In [27]:
SEED_PATH = '/Users/rachelbittner/Documents/HLL_Data/mdb_seeds_mel3/'
AUDIO_PATH = '/Datasets/MedleyDB_Mixes/'
ANNOT_PATH = '/Users/rachelbittner/Dropbox/MARL/repos/medleydb/Annotations/'

grid_search_trackids = [
    'MusicDelta_Country1', 'MusicDelta_Beatles', 'ClaraBerryAndWooldog_Boys',
    'MusicDelta_Rockabilly', 'Schubert_Erstarrung', 'AmarLal_SpringDay1'
]
file_triples = []
for tid in grid_search_trackids:
    audio_file = os.path.join(AUDIO_PATH, '{}_MIX.wav'.format(tid))
    seed_file = os.path.join(SEED_PATH, '{}_gt_seeds.csv'.format(tid))
    annot_file = os.path.join(ANNOT_PATH, '{}_ANNOTATIONS'.format(tid), '{}_MELODY3.csv'.format(tid))
    file_triples.append((audio_file, seed_file, annot_file))
print grid_search_trackids

['MusicDelta_Country1', 'MusicDelta_Beatles', 'ClaraBerryAndWooldog_Boys', 'MusicDelta_Rockabilly', 'Schubert_Erstarrung', 'AmarLal_SpringDay1']


In [15]:
row_data = []
n_iter = 1000

In [ ]:
for i in range(1497, 2000):
    print("running iteration {}".format(i))

    etr = parametrize_hll()

    recall = []
    precision = []
    accuracy = []
    for audio_file, seed_file, annot_file in file_triples:
        print("    {}".format(os.path.basename(audio_file)))

        ctr = run_hll(etr, audio_file, seed_file)
        est_time, est_freqs = ctr.to_multif0_format()
        ref_time, ref_freqs = load_melody3_multif0(annot_file)
        try:
            scores = mir_eval.multipitch.evaluate(np.array(ref_time), ref_freqs, np.array(est_time), est_freqs)
            recall.append(scores['Recall'])
            precision.append(scores['Precision'])
            accuracy.append(scores['Accuracy'])
        except ValueError:
            continue

    row = [
        np.mean(accuracy), np.std(accuracy), np.mean(precision), np.std(precision), np.mean(recall), np.std(recall),
        etr.amplitude_threshold, etr.f_cutoff, etr.min_contour_len_samples, etr.n_harmonics,
        etr.tracking_gain, etr.tracking_update_threshold
    ]
    row_data.append(row)


running iteration 1497
    MusicDelta_Country1_MIX.wav
    MusicDelta_Beatles_MIX.wav
    ClaraBerryAndWooldog_Boys_MIX.wav

In [82]:
columns = [
    'accuracy-mean', 'accuracy-std', 'precision-mean', 'precision-std', 'recall-mean', 'recall-std',
    'amplitude-threshold', 'f_cutoff', 'min_contour_len_samples', 'n_harmonics',
    'tracking_gain', 'tracking_update_threhold'
]
df = pd.DataFrame(row_data, columns=columns)
df.describe()

,accuracy-mean,accuracy-std,precision-mean,precision-std,recall-mean,recall-std,amplitude-threshold,f_cutoff,min_contour_len_samples,n_harmonics,tracking_gain,tracking_update_threhold
count,1497.000000,1497.000000,1497.000000,1497.000000,1497.000000,1497.000000,1497.000000,1497.0,1497.000000,1497.000000,1497.000000,1497.000000
mean,0.159217,0.041700,0.232839,0.051287,0.639933,0.072991,0.046177,30.0,45790.057448,4.104876,0.001072,64.148297
std,0.106580,0.025946,0.200528,0.034420,0.257150,0.031710,0.197717,0.0,90332.648090,1.466318,0.001053,80.117473
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.0,0.000000,1.000000,0.000080,5.000000
25%,0.055268,0.018346,0.069117,0.020345,0.537035,0.056534,0.000001,30.0,2205.000000,3.000000,0.000500,20.000000
50%,0.158711,0.044263,0.187177,0.050916,0.736141,0.072371,0.000010,30.0,11025.000000,4.000000,0.001000,50.000000
75%,0.250341,0.059637,0.325391,0.074644,0.832013,0.090121,0.001000,30.0,44100.000000,5.000000,0.001000,80.000000
max,0.391061,0.133173,0.879874,0.368985,0.941972,0.199055,1.000000,30.0,441000.000000,6.000000,0.005000,500.000000


In [83]:
best_accuracy_idx = df['accuracy-mean'].argmax()
df.ix[best_accuracy_idx]

accuracy-mean                  0.391061
accuracy-std                   0.111615
precision-mean                 0.577887
precision-std                  0.104368
recall-mean                    0.545232
recall-std                     0.146212
amplitude-threshold            0.001000
f_cutoff                      30.000000
min_contour_len_samples     2205.000000
n_harmonics                    5.000000
tracking_gain                  0.000300
tracking_update_threhold      30.000000
Name: 563, dtype: float64

In [84]:
best_recall_idx = df['recall-mean'].argmax()
df.ix[best_recall_idx]

accuracy-mean                   0.007368
accuracy-std                    0.003806
precision-mean                  0.007373
precision-std                   0.003811
recall-mean                     0.941972
recall-std                      0.020729
amplitude-threshold             0.000001
f_cutoff                       30.000000
min_contour_len_samples     44100.000000
n_harmonics                     6.000000
tracking_gain                   0.000800
tracking_update_threhold      500.000000
Name: 248, dtype: float64

In [88]:
beta = 3.0
rec = df['recall-mean']
pre = df['precision-mean']
weighted_f3 = (1.0 + beta**2.0)*(pre * rec)/((beta**2.0 * pre) + rec)
weighted_f3 = pd.DataFrame(weighted_f3, columns=['weighted-f3'])

beta = 5.0
weighted_f5 = (1.0 + beta**2.0)*(pre * rec)/((beta**2.0 * pre) + rec)
weighted_f5 = pd.DataFrame(weighted_f5, columns=['weighted-f5'])

beta=10.0
weighted_f10 = (1.0 + beta**2.0)*(pre * rec)/((beta**2.0 * pre) + rec)
weighted_f10 = pd.DataFrame(weighted_f10, columns=['weighted-f10'])

df2 = pd.concat([df, weighted_f3, weighted_f5, weighted_f10], axis=1)
print df2.ix[df2['weighted-f3'].argmax()]

accuracy-mean                   0.285223
accuracy-std                    0.038876
precision-mean                  0.321173
precision-std                   0.056803
recall-mean                     0.743237
recall-std                      0.065013
amplitude-threshold             0.000100
f_cutoff                       30.000000
min_contour_len_samples     11025.000000
n_harmonics                     4.000000
tracking_gain                   0.001000
tracking_update_threhold       40.000000
weighted-f3                     0.656910
weighted-f5                     0.707479
weighted-f10                    0.733691
Name: 163, dtype: float64


In [89]:
df2.to_csv('/Users/rachelbittner/Dropbox/MARL/repos/loopy/HLL_params_df.csv')